# Coursework 

**Initialize packages**

In [28]:
using JuMP, HiGHS, Plots, DataFrames, CSV, Statistics

**Reading Data**

In [ ]:
flex_avail = DataFrame(CSV.File("input_files/Flexible_load_availability.csv"));

In [129]:
flex_load = DataFrame(CSV.File("input_files/Flexible_load_data.csv"));

In [128]:
fuels = DataFrame(CSV.File("input_files/Fuels_data.csv")); 

In [127]:
generators = DataFrame(CSV.File("input_files/Generators_data.csv"));

In [126]:
generators_new = generators[generators.New_Build .== 1, :];

In [124]:
generators_old_opt = generators[generators.New_Build .== 0, :];

In [125]:
generators_old_cont = generators[generators.New_Build .== -1, :];

In [123]:
gen_var = DataFrame(CSV.File("input_files/Generators_variability.csv")) ;

In [122]:
load = DataFrame(CSV.File("input_files/Load_data.csv"));

In [121]:
network = DataFrame(CSV.File("input_files/Network.csv"));

In [120]:
# Extract row indices based on New_Build values
indices_new_build_1 = findall(generators.New_Build .== 1).+1;
indices_new_build_minus1 = findall(generators.New_Build .== -1).+1;
indices_new_build_0 = findall(generators.New_Build .== 0).+1;

In [117]:
var_old_optional = gen_var[:,indices_new_build_0];

In [118]:
var_old_continue = gen_var[:,indices_new_build_minus1];

In [119]:
var_new = gen_var[:,indices_new_build_1];

### Problem formulation
The objective function will be:
$\begin{align}
\min &\sum_{g \in NEW} FixedCost_g \times CAP_g + \sum_{g \in OLD continue} FixedCost_g \times CAP_g \\
&\quad\quad+\sum_{g \in OLD optional} FixedCost_g \times (ExistingCap_g - RET_g)&\\
&\quad\quad+ \sum_{g\in NEW}\sum_{h \in H} VarCost_g \times GEN_{g,h} + \sum_{g\in OLD continue}\sum_{h \in H} VarCost_g \times GEN_{g,h}\\[6pt]
&\quad\quad \sum_{g\in  OLD opional}\sum_{h \in H} VarCost_g \times GEN_{g,h} +  \sum_{h \in H} \sum_{z \in Z}NSECost \times NSE_h &\\
\end{align}$

s.t. the constraints
$\begin{align}
&\text{Supply demand balance constraint:}&\\[6pt]
&\sum_{g \in NEW} P_{g,h,z} + \sum_{g \in OLD optional} P_{g,h,z} + \sum_{g \in OLD continue} P_{g,h,z} + \sum_{s \in STOR} (DISCHARGE_{s,h,z} -CHARGE_{s,h,z} ) + &\\
& NSE_{h,z} - Demand_{h,z} + FShift_{h,z} -PShift_{h,z} = \sum_{j \in J_i} FLOW_{ij} & \forall \quad i \in \mathcal{N}\\
\end{align}$


$\begin{align}
&\text{Thermal limit constraint:}&\\[6pt]
& FLOW_{ij} \leq MaxFlow_{ij} & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
&\text{Flow direction constraint:}&\\[6pt]
& FLOW_{ij} = - FLOW_{ji} & \forall \quad i, j \in \mathcal{N} \\
\end{align}
$

$\begin{align}
&\text{Capacity constraint for new generators:}&\\[6pt]
& 0 \leq  GEN_{g,h} \leq CAP_g \times CF_{g,h} & \forall \quad g \in NEW, h \in H\\[6pt]
&\text{Capacity constraint for nuclear:}&\\[6pt]
& 0 \leq  GEN_{Nuclear,h} \leq CAP_g \times CF_{g,h} & \forall h \in H\\[6pt]
&\text{Capacity constraint for old continue generators:}&\\[6pt]
& 0 \leq  GEN_{g,h} \leq CAP_g \times CF_{g,h} & \forall \quad g \in Old Continue, h \in H\\[6pt]
&\text{Capacity constraint for existing optional generators:}&\\[6pt]
& 0 \leq  GEN_{g,h}  \leq (ExistingCap_g - RET_g)\times CF_{g,h} & \forall \quad g \in Old optional,  h \in H\\[6pt]
&\text{Capacity variable only for new build:}&\\[6pt]
& CAP_{g} \geq 0 & \forall \quad g \in NEW\\[6pt]
&\text{Retirement variable only for existing:}&\\[6pt]
& RET_{g} \geq 0 & \forall \quad g \in Old optional\\[6pt]
&\text{Generation and NSE variables:}&\\[6pt]
& GEN_{g,h} \geq 0 & \forall \quad g \in G \text{ \& } h \in H\\
& NSE_{h} \geq 0 & \forall \quad h \in H
\end{align}$

$\begin{align}
& GEN_{Nuclear,h} =  CAP_{Nuclear}           & \forall \quad  h \in H\\[6pt]
& 0 \leq CHARGE_{s,h} \leq CAP_s  & \forall \quad s \in STOR,  h \in H\\[6pt]
& 0 \leq DISCHARGE_{s,h} \leq CAP_s  & \forall \quad s \in STOR,  h \in H\\[6pt]
& \sum_{r \in RES} R_{r,h} +  (DISCHARGE_{BESS,h}-CHARGE_{BESS,h}) \leq 75\% * Demand_h & \forall \quad h \in H\\[6pt]
& SOC_{s,h}=SOC_{s,h-1} + \big(CHARGE_{s,h} * Eff_s - \frac{DISCHARGE_{s,h}}{Eff_s}\big)  & \forall \quad s \in S, h \in H \setminus {1} \\[6pt]
& 0 \leq SOC_{s,h} \leq SOC_{s}^{max}& \forall \quad s \in S, h \in H\\[6pt]
& SOC_{s,1} = SOC_{s,t_{end}} = 50\% *SOC_{s}^{max}& \forall \quad s \in S\\[6pt]
\end{align}$


In [134]:
@objective(Expansion_Model, Min,
    sum(generators[generators.NEW.==g,:Fixed_OM_cost_per_MWyr][1]*Existing_Cap_MW[g] ) + sum(generators[generators.OLD.==g,:])
    

)

LoadError: UndefVarError: `Expansion_Model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.